### utils 

In [3]:
from contentScoreShareUtils import * 

def add_timestamp_columns(stage1_raw):
    stage1_raw["date"] = stage1_raw["DateTime_UTC"].apply(lambda x: x[0:10]) ## 2022-11-01 00:00:02-06:00
    stage1_raw["hour"] = stage1_raw["DateTime_UTC"].apply(lambda x: x[11:13]) ## 2022-11-01 00:00:02-06:00
    # stage1_raw["minute"] = datetime.apply(lambda x: x.minute)
    return stage1_raw;

def convert_schema(stage1_raw):
    columns = pd.io.json.build_table_schema(stage1_raw)
    df_schema = {col["name"]:dtype_convert(col["type"]) for col in columns["fields"]}
    return df_schema

def dtype_convert(pandas_type):
    type = "Unknown"
    if pandas_type == "integer":
        type = INTEGER()
    elif pandas_type == "string":
        type = VARCHAR(255)
    elif pandas_type == "number":
        type = FLOAT(asdecimal=True)
    elif pandas_type == "boolean":
        type = BOOLEAN()
    return type

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def insert_with_progress(df, connection, table_name, insert_mode=False, dtype=None):
    chunksize = int(len(df) / 100) # 10%
    with tqdm(total=len(df)) as pbar:
        for i, cdf in enumerate(chunker(df, chunksize)):
            replace = "replace" if (i == 0 and insert_mode is False) else "append"
            cdf.to_sql(con=connection, name=table_name, dtype=dtype, if_exists=replace, index=False)
            pbar.update(chunksize)
                    
def load_csv_iter(glob_pattern="./target_mcvisid/*.csv"):
    files = sorted([file for file in glob(glob_pattern) if "aemRaw_keyColumns_2022" in file])
    for file in files:
        current = pd.read_csv(file)
        print("current file, ", file)
        yield file, current
        

### key filters 

In [4]:
def url_clean(x):
    x = re.sub(',|;', "", x) # clean special symbol
    x = re.sub("(#|.page|.aspx|.js|.php|adfs).*", "", x)  # clean tailing start with
    # x = re.sub("details.[\w-]*","details.{ID}", x) # grouping product details
    x = re.sub("com.cn", "com", x) # replace china area website URL
    # x = re.sub("%20", " ", x) # keep URL encoding
    
    # grouping different language
    x = re.sub("/$|(\.html)$|/[a-zA-Z]{2}$", "", x)  # remove html, etc /
    x = re.sub("/(global|go)/", "/", x)  
    x = re.sub(".com/[a-zA-Z]{2,3}/", ".com/", x)  
    x = re.sub("/[a-zA-Z]{2}[-_][a-zA-Z]{2,3}$", "", x)  # grouping homepage
    x = re.sub("/[a-zA-Z]{2}[-_][a-zA-Z]{2}[-_][a-zA-Z]{2}$", "", x)  # grouping search page
    x = re.sub("/[a-zA-Z]{2}[-_][a-zA-Z]{2,3}/|/[a-zA-Z]{2}/", "/", x)  # grouping different language
    
    return x

def url_filter(url_maps):
    url_maps = url_maps[(~url_maps["PageURL"].str.contains("file://.+")) & (url_maps["PageURL"].str.startswith("https://www.rockwellautomation"))] 
#     url_maps.query("('file://.+' not in PageURL) or https://www.rockwellautomation in PageURL") # query mode
    return url_maps
    
# x = "https://www.rockwellautomation.com/ja-jp/company/events/in-person-events/archive.html"
# x = "https://www.rockwellautomation.com/ja-jp/company/events/in-person-events/search/en-us-ab"
# x = "https://www.rockwellautomation.com/zh_TW"
# x = "https://www.rockwellautomation.com/hu-huhtml"
# x = "https://www.rockwellautomation.com/en-us/products/details.700-N400A1.html).html#adfdasf"
# x = "https://www.rockwellautomation.com/rockwellautomation/support/pcdc.page%7D%7D%7D/f0/fs20"
# x = "https://www.rockwellautomation.com/ja/fa.pagedfsdfa"
# x = "https://www.rockwellautomation.com/chl/overview.page"
# x = "https://www.rockwellautomation.com/adfs/oauth2/authorize"
x = "https://www.rockwellautomation.com/en-nz/"
url_clean(x)


'https://www.rockwellautomation.com'

### start ETL

In [3]:
VERSION = "v1.2"
table_name = "stage1_raw" + "_" + VERSION

refresh_table = True
mydb = create_engine('mysql+pymysql://rockwell:%s@localhost:3306/page_scoring' % quote('rockwell'))
testing = False
folder_dir = './aem_raw/'

schema = {'index': INTEGER(),
 'SessionVisitorId': VARCHAR(length=255),
 'VisitPageNumber': INTEGER(),
 'VisitNumber': INTEGER(),
 'NewVisit': BOOLEAN(),
 'EventList': VARCHAR(length=511),
 'DateTime_UTC': VARCHAR(length=255),
 'PageURL': VARCHAR(length=255),
 'VisitReferrer': VARCHAR(length=255),
 'VisitReferrerType': INTEGER(),
 'VisitorDomain': VARCHAR(length=255),
 'External_Audience': VARCHAR(length=255),
 'External_AudienceSegment': VARCHAR(length=255),
 'External_Industry': VARCHAR(length=255),
 'External_Website': VARCHAR(length=255),
 'EloquaContactId': VARCHAR(length=255),
 'EloquaGUID': FLOAT(asdecimal=True),
 'mcvisid': VARCHAR(length=255),
 'GeoCity': VARCHAR(length=255),
 'GeoCountry': VARCHAR(length=255),
 'GeoRegion': VARCHAR(length=255),
 'PDFurl': VARCHAR(length=255),
 'PDFpagecount': FLOAT(asdecimal=True),
 'BingeId': VARCHAR(length=255),
 'BingeCriticalScore': FLOAT(asdecimal=True),
 'BingeScoredAssetPath': VARCHAR(length=255),
 'BingeScoredAssetScore': FLOAT(asdecimal=True)}

In [4]:
# aem_raw = load_csv_batches(glob_pattern="./aem_raw/*.csv", num=None, keep="last", iterator=False)
aem_raws = load_csv_iter(glob_pattern=folder_dir+"*.csv")

if refresh_table is True:
    with mydb.connect() as connection:
        connection.execute(f"""DROP TABLE IF EXISTS {table_name};""")

### preload data, could update as a query
email_mcvisid = pd.read_csv("mcvisid_elqid_email_all.csv")
email_mcvisid, drop_mcvisid = email_cleanup(email_mcvisid, "EmailAddress")
positive_mcvisid = email_mcvisid["mcvisid"].drop_duplicates() ## all along the past, if mcvisid existed with corresponding elqid, then it is positive sigal



logs = []
for idx, (file, aem_raw) in enumerate(aem_raws):
    log = []
    log.append(file)
    stage1_raw = aem_raw #.dropna(subset=["BingeScoredAssetPath"]).reset_index(drop=True)
    if idx < 1:
        df_schema = convert_schema(stage1_raw) if not schema else schema
        print(df_schema)
    row1 = stage1_raw.shape[0]
    print(f"original raw rows: {row1}")
    log.append(row1)

    stage1_raw = stage1_raw[~stage1_raw["mcvisid"].isin(drop_mcvisid["mcvisid"])] # drop irrelevant mcvisid
    row2 = stage1_raw.shape[0]
    print(f"Drop irrelevant mcvisid (testing emails) with rows: {row1 - row2}")
    log.append(row1 - row2)

    ## deduplicate with hour granularity
    print("start dedup with considering only unique records in hours granularity...")
    stage1_raw = add_timestamp_columns(stage1_raw)    
    stage1_raw = stage1_raw.drop_duplicates(subset=["mcvisid","date", "hour", "PageURL"], keep="first")
    row3 = stage1_raw.shape[0]
    print(f"""Drop duplicate rows by ["mcvisid","date", "hour", "PageURL"]: {row2 - row3} ratio {(row2 - row3)/row2}""")
    log.append(row2 - row3)
    log.append((row2 - row3)/row2)

    ## get clean PageURL and drop irrelevant PageURL - inner join to drop unmatched rows
    url_maps = url_filter(stage1_raw[["PageURL"]].drop_duplicates())
    url_maps["clean_PageURL"] = url_maps["PageURL"].apply(lambda x: url_clean(x))
    # s = url_maps[url_maps["PageURL"].str.contains("details")].sample(100)
    stage1_raw = stage1_raw.merge(url_maps, on="PageURL")
    row4 = stage1_raw.shape[0]
    print(f"Drop irrelevant PageURL (by inner join) with rows: {row3 - row4}")
    log.append(row3 - row4)

    ## create labels
    stage1_raw["label"] = stage1_raw["mcvisid"].isin(positive_mcvisid) # mark pos and neg mcvisid
    row5 = stage1_raw.shape[0]
    print("stage1_raw: keep mcvisid rows: ", row5)
    print("stage1_raw: keep unique mcvisid: ", stage1_raw["mcvisid"].unique().shape[0])
    print("stage1_raw: keep unique pageURL: ", stage1_raw["clean_PageURL"].unique().shape[0])
    log.append(row5)
    log.append(stage1_raw["mcvisid"].unique().shape[0])
    log.append(stage1_raw["clean_PageURL"].unique().shape[0])
    
    try:
        insert_with_progress(stage1_raw, mydb, table_name=table_name, insert_mode=True, dtype=df_schema)
    except ValueError as vx:
        print(vx)
    except Exception as ex:
        print(ex)
    else:
        print("Table %s insert successfully." % table_name)
    
    logs.append(log)    
    if testing is True:
        break

with mydb.connect() as connection:
    connection.execute(f"""ALTER TABLE {table_name} ADD KEY(mcvisid(255));""")
    connection.execute(f"""ALTER TABLE {table_name} ADD KEY(EloquaContactId(255));""")

current file,  ./aem_raw/aemRaw_keyColumns_20220401-20220415.csv
{'index': INTEGER(), 'SessionVisitorId': VARCHAR(length=255), 'VisitPageNumber': INTEGER(), 'VisitNumber': INTEGER(), 'NewVisit': BOOLEAN(), 'EventList': VARCHAR(length=511), 'DateTime_UTC': VARCHAR(length=255), 'PageURL': VARCHAR(length=255), 'VisitReferrer': VARCHAR(length=255), 'VisitReferrerType': INTEGER(), 'VisitorDomain': VARCHAR(length=255), 'External_Audience': VARCHAR(length=255), 'External_AudienceSegment': VARCHAR(length=255), 'External_Industry': VARCHAR(length=255), 'External_Website': VARCHAR(length=255), 'EloquaContactId': VARCHAR(length=255), 'EloquaGUID': FLOAT(asdecimal=True), 'mcvisid': VARCHAR(length=255), 'GeoCity': VARCHAR(length=255), 'GeoCountry': VARCHAR(length=255), 'GeoRegion': VARCHAR(length=255), 'PDFurl': VARCHAR(length=255), 'PDFpagecount': FLOAT(asdecimal=True), 'BingeId': VARCHAR(length=255), 'BingeCriticalScore': FLOAT(asdecimal=True), 'BingeScoredAssetPath': VARCHAR(length=255), 'BingeS

1319666it [01:32, 14304.03it/s]                                                 


Table stage1_raw insert successfully.
current file,  ./aem_raw/aemRaw_keyColumns_20220415-20220430.csv
original raw rows: 5235230
Drop irrelevant mcvisid (testing emails) with rows: 276607
start dedup with considering only unique records in hours granularity...
Drop duplicate rows by ["mcvisid","date", "hour", "PageURL"]: 3563994 ratio 0.7187467165783726
Drop irrelevant PageURL (by inner join) with rows: 52367
stage1_raw: keep mcvisid rows:  1342262
stage1_raw: keep unique mcvisid:  689118
stage1_raw: keep unique pageURL:  263823


1355622it [01:38, 13695.28it/s]                                                 


Table stage1_raw insert successfully.
current file,  ./aem_raw/aemRaw_keyColumns_20220501-20220515.csv
original raw rows: 5495277
Drop irrelevant mcvisid (testing emails) with rows: 351812
start dedup with considering only unique records in hours granularity...
Drop duplicate rows by ["mcvisid","date", "hour", "PageURL"]: 4163824 ratio 0.8095367616966384
Drop irrelevant PageURL (by inner join) with rows: 36675
stage1_raw: keep mcvisid rows:  942966
stage1_raw: keep unique mcvisid:  366467
stage1_raw: keep unique pageURL:  24130


...

Table stage1_raw insert successfully.
current file,  ./aem_raw/aemRaw_keyColumns_20221101-20221115.csv
original raw rows: 7904562
Drop irrelevant mcvisid (testing emails) with rows: 411787
start dedup with considering only unique records in hours granularity...
Drop duplicate rows by ["mcvisid","date", "hour", "PageURL"]: 6329490 ratio 0.8447457717601289
Drop irrelevant PageURL (by inner join) with rows: 14374
stage1_raw: keep mcvisid rows:  1148911
stage1_raw: keep unique mcvisid:  462094
stage1_raw: keep unique pageURL:  25632


1160389it [01:18, 14719.28it/s]                                                 


Table stage1_raw insert successfully.
current file,  ./aem_raw/aemRaw_keyColumns_20221115-20221130.csv
original raw rows: 7898931
Drop irrelevant mcvisid (testing emails) with rows: 344666
start dedup with considering only unique records in hours granularity...
Drop duplicate rows by ["mcvisid","date", "hour", "PageURL"]: 6389764 ratio 0.8458485372170556
Drop irrelevant PageURL (by inner join) with rows: 8971
stage1_raw: keep mcvisid rows:  1155530
stage1_raw: keep unique mcvisid:  469041
stage1_raw: keep unique pageURL:  26315


1167055it [01:20, 14507.73it/s]                                                 


Table stage1_raw insert successfully.


In [5]:
logs_df = pd.DataFrame(logs, columns=[
            "file",
            "original raw rows",
            "Drop irrelevant mcvisid (testing emails) with rows",
            "Drop duplicate rows by ['mcvisid','date', 'hour', 'PageURL']",
            "Drop duplicate rows by ['mcvisid','date', 'hour', 'PageURL'] ratio",
            "Drop irrelevant PageURL (by inner join) with rows",
            "After ETL keep mcvisid rows",
            "After ETL keep unique mcvisid",
            "After ETL keep unique pageURL",])

logs_df.to_excel(f"{table_name}_table_ETL_logs.xlsx")

In [6]:
url_maps["isChanged"] = url_maps["clean_PageURL"]!=url_maps["PageURL"]
url_maps.to_csv("clean_PageURL_checking.csv", encoding='utf-8')

### store in local

In [7]:
usecols = [
    "mcvisid", "clean_PageURL", "label", "PDFurl", "BingeScoredAssetPath", "BingeId", "date", "hour",
    "VisitReferrer", "VisitorDomain", 'External_Audience',
    'External_AudienceSegment', 'External_Industry', 'External_Website',
    "EloquaContactId", "PageURL"
]

In [8]:
# table_name = "stage1_raw"
mydb = create_engine('mysql+pymysql://rockwell:%s@localhost:3306/page_scoring' % quote('rockwell'))
    
with mydb.connect() as connection:
    stage1_raw_2 = pd.read_sql_query(f'SELECT {", ".join(usecols)} FROM {table_name}',connection)
    
stage1_raw_2.to_csv("stage1_raw_v1.2.csv")